In [1]:
#Importing libraries
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, SimpleRNN, Embedding
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt


import tensorflow as tf
import pandas as pd
import tokenize, glob
import os, re, requests, tarfile, shutil, json, glob # For file manipualation only.

/Users/jahnavinp/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%tensorflow_version 2.x
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
!wget https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/Project_CodeNet.tar.gz

zsh:1: command not found: wget


In [4]:
# Experiment with tokenize library to see its use
with tokenize.open('data/p00002/Python/s005515634.py') as f:
    tokens = tokenize.generate_tokens(f.readline)
    for token in tokens:
        print(token)

TokenInfo(type=1 (NAME), string='while', start=(1, 0), end=(1, 5), line='while True:\n')
TokenInfo(type=1 (NAME), string='True', start=(1, 6), end=(1, 10), line='while True:\n')
TokenInfo(type=54 (OP), string=':', start=(1, 10), end=(1, 11), line='while True:\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(1, 11), end=(1, 12), line='while True:\n')
TokenInfo(type=5 (INDENT), string='    ', start=(2, 0), end=(2, 4), line='    try:\n')
TokenInfo(type=1 (NAME), string='try', start=(2, 4), end=(2, 7), line='    try:\n')
TokenInfo(type=54 (OP), string=':', start=(2, 7), end=(2, 8), line='    try:\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(2, 8), end=(2, 9), line='    try:\n')
TokenInfo(type=5 (INDENT), string='        ', start=(3, 0), end=(3, 8), line='        a, b = map(int, input().split())\n')
TokenInfo(type=1 (NAME), string='a', start=(3, 8), end=(3, 9), line='        a, b = map(int, input().split())\n')
TokenInfo(type=54 (OP), string=',', start=(3, 9), end=(3, 10), line=' 

In [5]:
all_files = glob.glob('data/*/Python/*.py')
print(len(all_files))
np.savetxt('/all_py_files.txt', X = all_files, delimiter=',', comments='', fmt='%s')

1600


OSError: [Errno 30] Read-only file system: '/all_py_files.txt'

In [6]:
all_files_mini = glob.glob('data/*/Python/*.py')
np.savetxt('all_py_files.txt', X = all_files_mini, delimiter=',', comments='', fmt='%s')
len(all_files_mini)

1600

In [7]:
def tokenize_and_save(dir_name):
  tokenizerD = Tokenizer(
    num_words=10000,
    filters='',
    lower=False, split=' ', char_level=False, oov_token="<UNK>",
    document_count=0
  )
  
  # Assumes a file with all_py_files.txt name
  file_index = 1
  # with open(dir_name + 'all_py_files.txt') as dir_f:
  #   for py_file in dir_f.readlines():
  for py_file in all_files:
    file_index = file_index + 1
    with tokenize.open(py_file.strip()) as f:
      try:
        tokens = [tok.string for tok in tokenize.generate_tokens(f.readline)]
        tokenizerD.fit_on_texts(tokens)
      except:
        pass 

      if file_index % 2000 == 0:
        print(str(file_index) + " python files tokenized with #tokens: " + str(len(tokenizerD.word_index)))
        tokenizer_json = tokenizerD.to_json()
        with open(dir_name + 'tokenizer.json', 'w') as f:
          json.dump(tokenizer_json, f)

  # Save final version
  tokenizer_json = tokenizerD.to_json()
  with open(dir_name + 'tokenizer.json', 'w') as f:
    json.dump(tokenizer_json, f)

  print('Total tokenizer vocabulary {}'.format(len(tokenizerD.word_index)))


tokenize_and_save('data')

Total tokenizer vocabulary 2091


In [12]:
model1 = tf.keras.models.load_model("LSTM_model_mini_18Apr2022.hdf5")

In [11]:
model2 = keras.models.load_model("RNN_model_mini_18Apr2022.hdf5")